In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from sklearn.cluster import DBSCAN

In [2]:
bb=pd.read_csv('bkk_budget_ocr.csv',index_col=0)
bb=bb[~bb['pagenum'].isna()]

/Users/napatswift/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# number of document
len(bb.pdf.unique())

385

In [4]:
bb['pagenum'] = bb['pagenum'].astype(int)
bb['text'] = bb['text'].astype(str)

select test set

In [5]:
interested_doc = [
    'pdf/61/A20171003163303.pdf', 
    'pdf/61/A20171003164329.pdf', 
    'pdf/61/A20171003170601.pdf', 
    'pdf/62/A20181002170334.pdf',
    'pdf/64/topic2787_2020_10_06_10_18_31.pdf',
    'pdf/64/topic2787_2020_10_06_10_18_51.pdf']

create total sum of the document for validation

In [62]:
answers = {
    'pdf/61/A20171003163303.pdf': {'sum':691_229_200, 'start': 3},
    'pdf/61/A20171003164329.pdf': {'sum': 386_943_640, 'start': 5},
    'pdf/61/A20171003170601.pdf': {'sum': 365_299_840, 'start': 5},
    'pdf/62/A20181002170334.pdf': {'sum': 357_892_000, 'start': 5},
    'pdf/64/topic2787_2020_10_06_10_18_31.pdf': {'sum': 424_674_000, 'start': 5},
    'pdf/64/topic2787_2020_10_06_10_18_51.pdf': {'sum': 398_016_000, 'start': 5},
    }

remove unwanted document (just for develop)

In [7]:
bb=bb[bb.pdf.isin(interested_doc)]

fix text by dict

In [8]:
chr_fix_pair={'ำ': 'า','/า': 'ำ',}

In [9]:
def fix_pdf_text(old_text):
    new_text = old_text.strip()
    for o,n in chr_fix_pair.items():
        new_text=new_text.replace(o,n)
    return new_text

bb=bb.assign(fix_text=bb.text.apply(fix_pdf_text))

replace text from pdf with OCR text

In [10]:
bb.loc[~bb['ocr-text'].isna(), 'fix_text'] = bb[~bb['ocr-text'].isna()]['ocr-text']

In [58]:
max_dist = 0.7
for g, idx in tqdm(bb.groupby('image_path').groups.items()):
    out = bb.loc[idx][['x0', 'x1', 'y0', 'y1']]
    out['h'] = out.y1 - out.y0
    num_bboxes = len(idx)
    idx2idx = {j:i for i,j in enumerate(idx)}
    n_dist_mat = np.ones((num_bboxes, num_bboxes))
    dist_mat = np.ones((num_bboxes, num_bboxes))
    
    for i in range(num_bboxes):
        a = out.iloc[i]
        _out=out.loc[idx[i:]]
        _skip = (a.y0 > _out.y1) | (_out.y0 > a.y1)
        _out = _out[~_skip]
        intersection = np.minimum(a.y1, _out.y1) - np.maximum(a.y0, _out.y0)
        union = a.h + _out.h - intersection
        iou = np.clip(intersection / (union + 1e-10), 0, 1)
        
        idicies = [idx2idx[i] for i in iou.index]

        n_dist_mat[i, idicies] = 1-iou.values
        n_dist_mat[idicies, i] = 1-iou.values
        
        continue
        iou_list=[]
        for j in range(i, num_bboxes):
            b = out.iloc[j]

            if a.y0 > b.y1 or b.y0 > a.y1:
                continue

            intersection = min(a.y1, b.y1) - max(a.y0, b.y0)
            union = a.h + b.h - intersection
            iou = np.clip(intersection / union if union > 0 else 0, 0, 1)
            iou_list.append(iou)
            dist_mat[i, j] = dist_mat[j, i] = 1 - iou

    dbscan = DBSCAN(eps=max_dist, min_samples=0, metric='precomputed').fit(n_dist_mat)
    clustered = defaultdict(list)
    bb.loc[idx, 'line_label'] = dbscan.labels_
    
    continue
    for i, cluster_id in enumerate(dbscan.labels_):
        if cluster_id == -1:
            continue
        clustered[cluster_id].append(idx[i])

    res = sorted(clustered.values(), key=lambda line: [bb.loc[det].y1 / 2 for det in line])
    pinrt([[bb.loc[b].fix_text for b in line] for line in res])

100%|█████████████████████████████████████████| 221/221 [00:55<00:00,  4.02it/s]


In [25]:
# dist_mat,n_dist_mat

In [55]:
bb.loc[:, 'line_label'] = bb.line_label.astype(int)

In [60]:
# bb[~bb.line_label.isna()]

In [61]:
bb[~bb.line_label.isna()][['fix_text', 'line_label']].values[:100]

array([['223', 0],
       ['สำนักงานเขตทุ่งครุ', 1],
       ['งบประมาณรายจ่าย', 2],
       ['ปี', 3],
       ['2560', 3],
       ['348,560,900', 3],
       ['บาท', 3],
       ['ปี', 4],
       ['2561', 4],
       ['365,299,840', 4],
       ['บาท', 4],
       ['รายจ่ายประจำ', 5],
       ['365,299,840', 5],
       ['บาท', 5],
       ['รหัส', 6],
       ['รายจ่ายประจำ', 6],
       ['01', 7],
       ['ก.', 7],
       ['ด้านการบริหารทั่วไป', 7],
       ['36,289,140', 7],
       ['บาท', 7],
       ['0102', 8],
       ['1.', 8],
       ['แผนงานบริหารงานปกครอง', 8],
       ['และทะเบียน', 9],
       ['26,964,640', 9],
       ['บาท', 9],
       ['0102002', 10],
       ['1.1', 10],
       ['งานปกครอง', 10],
       ['20,356,140', 10],
       ['บาท', 10],
       ['0102003', 11],
       ['1.2', 11],
       ['งานทะเบียน', 11],
       ['6,608,500', 11],
       ['บาท', 11],
       ['0103', 12],
       ['2.', 12],
       ['แผนงานบริหารการคลัง', 12],
       ['9,324,500', 12],
       ['บาท', 12],
       [